In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc

# new:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

Take the data from Lino's preprocessing script

In [93]:
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

In [94]:
print("X info:")
X.info()
print("\n y info:")
y.info()

X info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76822 entries, 0 to 76821
Columns: 495 entries, age to job_end_2019
dtypes: bool(259), float64(236)
memory usage: 157.3 MB

 y info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76822 entries, 0 to 76821
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sphus   76822 non-null  object
dtypes: object(1)
memory usage: 600.3+ KB


In [95]:
y.head

<bound method NDFrame.head of               sphus
0           5. Poor
1           4. Fair
2           4. Fair
3           3. Good
4      2. Very good
...             ...
76817       4. Fair
76818       4. Fair
76819       3. Good
76820       4. Fair
76821       3. Good

[76822 rows x 1 columns]>

OK, need to convert this to numeric to make it into tensor (later will need to map it back to categorical to get meaningful answers)

In [96]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y['sphus'] = label_encoder.fit_transform(y['sphus'])
print(y)

       sphus
0          4
1          3
2          3
3          2
4          1
...      ...
76817      3
76818      3
76819      2
76820      3
76821      2

[76822 rows x 1 columns]


In [97]:
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state = 123)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = test_ratio / (test_ratio + validation_ratio), random_state = 10117)

n_train = X_train.shape[0]
n_val = X_val.shape[0]
n_test = X_test.shape[0]
n = n_train + n_val + n_test

print((n_train / n), (n_val / n), (n_test / n)) 

0.6999947931582099 0.14999609486865742 0.15000911197313269


In [98]:
# Check for NaNs in the training data
if X_train.isna().any().any() or y_train.isna().any().any():
    print("NaNs in train data")
else:
    print("No NaNs in train data")

# Check for infinities in the training data
if (X_train == np.inf).any().any() or (y_train == np.inf).any().any():
    print("Infinities in train data")
else:
    print("No Infinities in train data")

No NaNs in train data
No Infinities in train data


# Impute & scale

In [99]:
# IMPUTING:

# create copies of original
X_train_imputed = X_train.copy()
X_test_imputed = X_test.copy()
X_val_imputed = X_val.copy()

# non numeric
# Select non-numeric columns (booleans)
non_numeric_columns_train = X_train_imputed.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_test = X_test_imputed.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_val = X_val_imputed.select_dtypes(include=['object', 'bool']).columns


# Impute non-numeric columns with the string "missing"
X_train_imputed[non_numeric_columns_train] = X_train_imputed[non_numeric_columns_train].fillna("missing")
X_test_imputed[non_numeric_columns_test] = X_test_imputed[non_numeric_columns_test].fillna("missing")
X_val_imputed[non_numeric_columns_val] = X_val_imputed[non_numeric_columns_val].fillna("missing")


# numeric
# Select numeric columns
numeric_columns_train = X_train_imputed.select_dtypes(include=[np.number]).columns
numeric_columns_test = X_test_imputed.select_dtypes(include=[np.number]).columns
numeric_columns_val = X_val_imputed.select_dtypes(include=[np.number]).columns

# Initialize the imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')

# Apply imputation to numeric columns in training data (fit & transform)
X_train_imputed[numeric_columns_train] = numeric_imputer.fit_transform(X_train_imputed[numeric_columns_train])

# Apply imputation to numeric columns in testing / val data using the *same* imputer (only transform)
X_test_imputed[numeric_columns_test] = numeric_imputer.transform(X_test_imputed[numeric_columns_test])
X_val_imputed[numeric_columns_val] = numeric_imputer.transform(X_val_imputed[numeric_columns_val])


In [100]:
# SCALING

# initialise
scaler = StandardScaler()

# Scale training data (fit and transform)
X_train_imputed_scaled = X_train_imputed.copy()
X_train_imputed_scaled[numeric_columns_train] = scaler.fit_transform(X_train_imputed[numeric_columns_train])

# Scale testing & validation data (only transform)
X_test_inputed_scaled = X_test_imputed.copy()
X_test_inputed_scaled[numeric_columns_test] = scaler.transform(X_test_imputed[numeric_columns_test])

X_val_imputed_scaled = X_val_imputed.copy()
X_val_imputed_scaled[numeric_columns_test] = scaler.transform(X_val_imputed[numeric_columns_test])

# Convert to Tensors

In [101]:
# Convert feature DataFrames to NumPy arrays and then to tensors
X_train_torch = torch.tensor(X_train_imputed_scaled.to_numpy().astype(np.float32))
X_val_torch = torch.tensor(X_val_imputed_scaled.to_numpy().astype(np.float32))
X_test_torch = torch.tensor(X_test_inputed_scaled.to_numpy().astype(np.float32))

# Convert label Series to tensors and unsqueeze to add an extra dimension
y_train_torch = torch.tensor(y_train.to_numpy().astype(np.float32)).unsqueeze(1)
y_val_torch = torch.tensor(y_val.to_numpy().astype(np.float32)).unsqueeze(1)
y_test_torch = torch.tensor(y_test.to_numpy().astype(np.float32)).unsqueeze(1)

since we're doing classification we do NOT need to scale the y-label series right?

In [102]:
# DataLoader instances

# Ensure targets are Long tensors and 1D
train_dataset = TensorDataset(X_train_torch, y_train_torch.long().view(-1))
val_dataset = TensorDataset(X_val_torch, y_val_torch.long().view(-1))
test_dataset = TensorDataset(X_test_torch, y_test_torch.long().view(-1))

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


below: had errors with targets being out of bound

In [103]:
try:
    # number of output classes required
    num_classes = len(torch.unique(y_train_torch))

    # Ensure all target values are within the valid range
    assert torch.all((y_train_torch >= 0) & (y_train_torch < num_classes)), "Train targets out of range"
    assert torch.all((y_val_torch >= 0) & (y_val_torch < num_classes)), "Validation targets out of range"
    assert torch.all((y_test_torch >= 0) & (y_test_torch < num_classes)), "Test targets out of range"
    print("All OK")
except AssertionError as e:
    print(e)

All OK


# Define Model Architectures

### Basic linear, 3 layered (???) NN

In [104]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_features, hidden_size, output_features):
        super(NeuralNetwork, self).__init__()
        self.hidden = nn.Linear(input_features, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_features)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return F.softmax(x, dim=1)

### Deep Model:

- Batch normalization helps in normalising  inputs of each layer, reducing internal covariate shift. 
- Same as before: nonlinear ReLU activation function ensures.

In [105]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super(DeepNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_features, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.layer2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.layer3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.output = nn.Linear(128, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.layer1(x)))
        x = self.relu(self.bn2(self.layer2(x)))
        x = self.relu(self.bn3(self.layer3(x)))
        x = self.output(x)
        return x

### Dropout model
- after 1st activation layer, using a dropout rate of 50% to prevent overfitting - works by randomly deactivating half of the output units during training. 
- LeakyReLU: against issue of dying nodes (neurons) through small, positive gradient for negative inputs

In [106]:
class DropoutNeuralNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super(DropoutNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_features, 500)
        self.layer2 = nn.Linear(500, 100)
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(100, 1)
        self.leaky_relu = nn.LeakyReLU(0.01)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.output(x)
        return x

### Combo Model

- integrates both dropout regularization and batch normalization across three layers 
- intended to enhance training stability and prevent overfitting
- employs Leaky ReLU as the activation function again to ensure consistent small gradients

In [107]:
class CombinedNeuralNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super(CombinedNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_features, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.25)  # Moderate dropout
        self.layer2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.25)
        self.layer3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.output = nn.Linear(128, 1)
        self.leaky_relu = nn.LeakyReLU(0.01)

    def forward(self, x):
        x = self.dropout1(self.leaky_relu(self.bn1(self.layer1(x))))
        x = self.dropout2(self.leaky_relu(self.bn2(self.layer2(x))))
        x = self.leaky_relu(self.bn3(self.layer3(x)))
        x = self.output(x)

        return x

### Mixed Activation Functions

- uses different activation functions at different layers to capture different types of nonlinear relationships at various levels of abstraction
-  idea is to enable the model to learn more *diverse* representations by varying the transformations applied to inputs / intermediate features.

In [108]:
class MixedActivationNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super(MixedActivationNetwork, self).__init__()
        self.layer1 = nn.Linear(input_features, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 128)
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.layer3 = nn.Linear(128, 64)
        self.elu = nn.ELU()
        self.output_layer = nn.Linear(64, output_features)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.leaky_relu(self.layer2(x))
        x = self.elu(self.layer3(x))
        x = self.output_layer(x)
        return x

### ResNet

This came from ChatGPT, I didn't know a ResNet before:

"Employs principles of a Residual Network (ResNet) architecture adapted for regression tasks. It is designed with an initial linear block to project input features into a 64-dimensional space, followed by a series of residual blocks that apply transformations and integrate the original input via skip connections to enhance learning without degradation, culminating with a final linear layer to produce a scalar output"

- main point: residual blocks enable deeper networks by adding shortcut connections that bypass one or more layers, aiming to address the problem of vanishing gradients in deep neural networks

In [109]:
# build module
class ResidualBlock(nn.Module):
    def __init__(self, input_features, output_features):
        super(ResidualBlock, self).__init__()
        self.linear1 = nn.Linear(input_features, output_features)
        self.bn1 = nn.BatchNorm1d(output_features)
        self.relu = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(output_features, output_features)
        self.bn2 = nn.BatchNorm1d(output_features)

    def forward(self, x):
        identity = x
        out = self.linear1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.bn2(out)

        out += identity  
        out = self.relu(out)

        return out
    

# build model
class ResNetRegression(nn.Module):
    def __init__(self, input_features, block, num_blocks, output_features=1):
        super(ResNetRegression, self).__init__()
        self.num_blocks = num_blocks
        self.input_features = input_features
        self.current_features = 64  # here is initial number of features
        self.init_block = nn.Sequential(
            nn.Linear(input_features, self.current_features),
            nn.BatchNorm1d(self.current_features),
            nn.ReLU(inplace=True)
        )
        self.blocks = self._make_layer(block, 64, num_blocks)
        self.final_layer = nn.Linear(64, output_features)

    def _make_layer(self, block, output_features, num_blocks):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(self.current_features, output_features))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.init_block(x)
        x = self.blocks(x)
        x = self.final_layer(x)
        return x

### Dynamic Routing Network

This was purely from ChatGPT again, but the ideas is a "routing" mechanism that dynamically decides how much of each layer's output should contribute to the next layer's input.

- uses gating layers to selectively control the flow of information through the network by multiplying the output of a linear transformation of the input (x_transformed) by a gating signal (gate). 
- gating mechanism is a sigmoid function, which scales the linearly transformed input before passing it through gated layers
- as before, residual blocks enable deeper networks by adding shortcut connections that bypass one or more layers, aiming to address the problem of vanishing gradients in deep neural networks

In [110]:
# build module 
class GatingLayer(nn.Module):
    def __init__(self, input_features, output_features):
        super(GatingLayer, self).__init__()
        self.layer = nn.Linear(input_features, output_features)
        self.gate = nn.Sequential(
            nn.Linear(input_features, output_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_transformed = self.layer(x)
        gate = self.gate(x)
        return x_transformed * gate

# build model
class DynamicRoutingNetwork(nn.Module):
    def __init__(self, input_features, output_features):
        super(DynamicRoutingNetwork, self).__init__()
        self.layer1 = GatingLayer(input_features, 128)
        self.layer2 = GatingLayer(128, 64)
        self.layer3 = GatingLayer(64, 64)
        self.output_layer = nn.Linear(64, output_features)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output_layer(x)
        return x

# Initialise Models

In [111]:
# # initialize models

input_features = X_train_torch.shape[1]
hidden_size = 64  
output_features = num_classes  

# Initialize the neural network
model = NeuralNetwork(input_features, hidden_size, output_features)

basic_model = NeuralNetwork(input_features, hidden_size, output_features)
basic_model.name = "basic"

# deep_model = DeepNeuralNetwork(input_features=X_train_torch.shape[1], output_features=num_classes)
# deep_model.name = "Deep"

# dropout_model = DropoutNeuralNetwork(input_features=X_train_torch.shape[1])
# dropout_model.name = "Dropout"

# combo_model = CombinedNeuralNetwork(input_features=X_train_torch.shape[1])
# combo_model.name = "Combo"

# mixed_model = CombinedNeuralNetwork(input_features=X_train_torch.shape[1])
# mixed_model.name = "Mixed"

# ResNet_model = ResNetRegression(input_features=X_train_torch.shape[1], block=ResidualBlock, num_blocks=3)
# ResNet_model.name = "ResNet"

# routing_model = ResNetRegression(input_features=X_train_torch.shape[1], block=ResidualBlock, num_blocks=3)
# routing_model.name = "routing"


# Write functions for full workflow

### 1) A function to train the model over one epoch

In [114]:
def train_one_epoch(train_loader, model, loss_fn, optimizer):
    model.train()  
    total_loss = 0.0
    
    for batch in train_loader:
        inputs, targets = batch
        
        # Ensure targets are Long tensor and 1D
        targets = targets.long().view(-1)
        
        optimizer.zero_grad()  # Zero previous gradients

        # Forward pass: compute model output
        outputs = model(inputs)
        
        # Compute loss
        loss = loss_fn(outputs, targets)

        # Backward pass: compute gradient of loss wrt to model params
        loss.backward()  # Backpropagation
        
        # Update the model parameters
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()

    # Return the average loss for this epoch
    average_loss = total_loss / len(train_loader)
    return average_loss


### 2) A function to evaluate error on entire val / test set


In [115]:
def evaluate_model(data_loader, model, loss_fn):
    model.eval()  
    total_loss = 0.0

    with torch.no_grad():  # Disable gradient computation during evaluation
        for batch in data_loader:
            inputs, targets = batch

            # Ensure targets are Long tensor and 1D
            targets = targets.long().view(-1)

            # Forward pass
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_fn(outputs, targets)

            # Aggregate the loss
            total_loss += loss.item()
            
    # Return average loss for the entire dataset
    average_loss = total_loss / len(data_loader) 
    return average_loss


### 3) A function to integrate training and validation, returning the best model

In [116]:
def train_and_validate(train_loader, val_loader, model, epochs, optimizer, loss_fn):
    best_val_loss = float('inf')
    best_model_state = None
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Train the model for one epoch
        train_loss = train_one_epoch(train_loader, model, loss_fn, optimizer)
        train_losses.append(train_loss)

        # Evaluate the model on the validation set
        val_loss = evaluate_model(val_loader, model, loss_fn)
        val_losses.append(val_loss)

        # Check if this is the smallest so far and save associated model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    # Load the best model state
    model.load_state_dict(best_model_state)
    
    return model, train_losses, val_losses

### 4) A function to get predictions from a given model

In [117]:
def get_predictions(model, data_loader):
    model.eval()  # Set model to evaluation mode
    predictions = []

    with torch.no_grad():  # Disable gradient computation
        for inputs, _ in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
            predictions.extend(predicted.cpu().numpy())  # Convert to numpy array and extend the list
    
    return np.array(predictions)  # Convert the list to a numpy array



### 4) A function to produce classification report

In [118]:
from sklearn.metrics import classification_report

def generate_classification_report(model, test_loader):
    predictions = get_predictions(model, test_loader)
    
    # Extract true labels from test_loader
    true_labels = []
    for _, targets in test_loader:
        true_labels.extend(targets.cpu().numpy())
    
    # Generate and print classification report
    report = classification_report(true_labels, predictions, target_names=label_encoder.classes_)
    print(f"Classification Report for {model.__class__.__name__}:")
    print(report)
    
    return report

### 5) A final workflow function - brings it all together

In [119]:
def run_workflow(models, train_loader, val_loader, test_loader, epochs=20):
    best_val_loss = float('inf')
    best_model = None
    results = []

    for model in models:
        print(f"Training model: {model.__class__.__name__}")
        
        # Define loss function and optimizer 
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Train and validate the model
        trained_model, train_losses, val_losses = train_and_validate(
            train_loader, val_loader, model, epochs, optimizer, loss_fn
        )

        # Evaluate on the test set
        test_loss = evaluate_model(test_loader, trained_model, loss_fn)
        
        classification_report_str = generate_classification_report(trained_model, test_loader)
        
        results.append({
            'model_name': model.__class__.__name__,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'test_loss': test_loss,
            'classification_report': classification_report_str
        })
        
        # Track the best model based on validation loss
        if min(val_losses) < best_val_loss:
            best_val_loss = min(val_losses)
            best_model = trained_model
        
        print(f"Completed training for model: {model.__class__.__name__}")

    return best_model, results


Summary: need to provide following parameters to run_workflow():
- models (a vector of models???), 
- train_loader, 
- val_loader, 
- test_loader, 
- (epochs)

and it will return:
- best_model,
- results

# Run Comparative Training & Validating

Reminder: need to provide following parameters to run_workflow():
Summary: need to provide following parameters to run_workflow():
- models (a vector of models???), 
- train_loader, 
- val_loader, 
- test_loader, 
- (epochs)

and it will return:
- best_model,
- results

In [120]:
#test_model, test_results = run_workflow(deep_model, train_loader, val_loader, test_loader, epochs=20)

In [121]:
# Example models
models = [
    NeuralNetwork(input_features=X_train_torch.shape[1], output_features=num_classes, hidden_size=10),
    NeuralNetwork(input_features=X_train_torch.shape[1], output_features=num_classes, hidden_size=100),
    NeuralNetwork(input_features=X_train_torch.shape[1], output_features=num_classes, hidden_size=1000)
]

# Run 
best_model, results = run_workflow(models, train_loader, val_loader, test_loader, epochs=20)

# Print results summary
for result in results:
    print(f"Model: {result['model_name']}")
    print(f"Test Loss: {result['test_loss']:.4f}")
    print(f"Classification Report:\n{result['classification_report']}")

Training model: NeuralNetwork
Epoch 1/20, Train Loss: 1.4221, Validation Loss: 1.4031
Epoch 2/20, Train Loss: 1.3870, Validation Loss: 1.3787
Epoch 3/20, Train Loss: 1.3718, Validation Loss: 1.3749
Epoch 4/20, Train Loss: 1.3665, Validation Loss: 1.3723
Epoch 5/20, Train Loss: 1.3631, Validation Loss: 1.3726
Epoch 6/20, Train Loss: 1.3609, Validation Loss: 1.3706
Epoch 7/20, Train Loss: 1.3593, Validation Loss: 1.3679
Epoch 8/20, Train Loss: 1.3575, Validation Loss: 1.3737
Epoch 9/20, Train Loss: 1.3567, Validation Loss: 1.3699
Epoch 10/20, Train Loss: 1.3549, Validation Loss: 1.3686
Epoch 11/20, Train Loss: 1.3535, Validation Loss: 1.3680
Epoch 12/20, Train Loss: 1.3528, Validation Loss: 1.3708
Epoch 13/20, Train Loss: 1.3520, Validation Loss: 1.3687
Epoch 14/20, Train Loss: 1.3511, Validation Loss: 1.3672
Epoch 15/20, Train Loss: 1.3498, Validation Loss: 1.3674
Epoch 16/20, Train Loss: 1.3494, Validation Loss: 1.3691
Epoch 17/20, Train Loss: 1.3490, Validation Loss: 1.3686
Epoch 18/2

/Users/henrybaker/miniconda3/envs/MLLab/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrybaker/miniconda3/envs/MLLab/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrybaker/miniconda3/envs/MLLab/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

Classification Report for NeuralNetwork:
              precision    recall  f1-score   support

1. Excellent       0.00      0.00      0.00       701
2. Very good       0.43      0.40      0.42      1769
     3. Good       0.53      0.62      0.57      4226
     4. Fair       0.51      0.58      0.54      3368
     5. Poor       0.65      0.50      0.57      1460

    accuracy                           0.52     11524
   macro avg       0.43      0.42      0.42     11524
weighted avg       0.49      0.52      0.51     11524

Completed training for model: NeuralNetwork
Training model: NeuralNetwork
Epoch 1/20, Train Loss: 1.4078, Validation Loss: 1.3888
Epoch 2/20, Train Loss: 1.3757, Validation Loss: 1.3814
Epoch 3/20, Train Loss: 1.3687, Validation Loss: 1.3746
Epoch 4/20, Train Loss: 1.3663, Validation Loss: 1.3761
Epoch 5/20, Train Loss: 1.3617, Validation Loss: 1.3830
Epoch 6/20, Train Loss: 1.3598, Validation Loss: 1.3739
Epoch 7/20, Train Loss: 1.3586, Validation Loss: 1.3696


KeyboardInterrupt: 